## RusVectōrēs: семантические модели для русского языка

#### Елизавета Кузьменко, Андрей Кутузов

В этом тьюториале мы рассмотрим возможности использования веб-сервиса RusVectōrēs и векторных семантических моделей, которые этот веб-сервис предоставляет пользователям. Наша задача -- от "сырого" текста (т.е. текста без всякой предварительной обработки) прийти к данным, которые мы можем передать векторной модели и получить от неё интересующий нас результат.

Тьюториал состоит из трех частей:
* в первой части мы научимся осуществлять предобработку текстовых файлов так, чтобы в дальнейшем они могли использованы в качестве входных данных для моделей RusVectōrēs.
* во второй части мы научимся работать с векторными моделями и выполнять простые операции над векторами слов, такие как "найти семантические аналоги", "сложить вектора двух слов", "вычислить коэффициент близости между двумя векторами слов". 
* в третьей части мы научимся обращаться к сервису RusVectōrēs через API.

Мы рекомендуем использовать **Python3**, работоспособность тьюториала для **Python2** не гарантируется.

## 1. Предобработка текстовых данных

Функциональность **RusVectōrēs** позволяет пользователям делать запрос к моделям [с одним конкретным словом](https://rusvectores.org/ru/similar/) или [с несколькими словами](https://rusvectores.org/ru/calculator/). С помощью сервиса можно также анализировать отношения между [бóльшим количеством слов](https://rusvectores.org/ru/visual/). Но что делать, если вы хотите обработать очень большую коллекцию текстов или ваша задача не решается при помощи конкретных единичных запросов к серверу, которые можно сделать вручную?

В этом случае можно скачать одну из наших [моделей](https://rusvectores.org/ru/models/), а затем обрабатывать с её помощью тексты локально на вашем компьютере. Однако в этом случае необходимо, чтобы данные, которые подаются на вход модели, были в том же формате, что и данные, на которых эта модель была натренирована.

Вы можете использовать наши готовые скрипты, чтобы из сырого текста получить текст в формате, который можно подавать на вход модели. Скрипты лежат [здесь](https://github.com/akutuzov/webvectors/tree/master/preprocessing). Как следует из их названия, один из скриптов использует для предобработки UDPipe, а другой Mystem. Оба скрипта используют стандартные потоки ввода и вывода, принимают на вход текст, выдают тот же текст, только лемматизированный и с частеречными тэгами. Если же вы хотите детально во всем разобраться и понять, например, в чем разница между UDPipe и Mystem, то читайте далее :)

Предобработка текстов для тренировки моделей осуществлялась следующим образом:
* лемматизация и удаление стоп-слов;
* приведение лемм к нижнему регистру;
* добавление частеречного тэга для каждого слова.

Особого внимания заслуживает последний пункт предобработки. Как можно видеть из таблицы с описанием моделей, частеречные тэги для слов в различных моделях принадлежат к разным тагсетам. Первые модели использовали [тагсет **Mystem**](https://tech.yandex.ru/mystem/doc/grammemes-values-docpage/), затем мы перешли на [**Universal POS tags**](https://universaldependencies.org/u/pos/). В моделях на базе [**fastText**](https://fasttext.cc/) частеречные тэги не используются вовсе.

Давайте попробуем воссоздать процесс предобработки текста на примере рассказа [О. Генри "Русские соболя"](https://rusvectores.org/static/henry_sobolya.txt). Для предобработки мы предлагаем использовать [*UDPipe*](https://ufal.mff.cuni.cz/udpipe), чтобы сразу получить частеречную разметку в виде Universal POS-tags. Сначала установим обертку *UDPipe* для Python:

`pip install ufal.udpipe`

*UDPipe* использует предобученные модели для лемматизации и тэггинга. Вы можете использовать [нашу модель](https://rusvectores.org/static/models/udpipe_syntagrus.model) или обучить свою. 

Чтобы загружать файлы, можно использовать реализацию wget в виде питоновской библиотеки:

`pip install wget`

In [1]:
import wget

udpipe_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
text_url = 'https://rusvectores.org/static/henry_sobolya.txt'

modelfile = wget.download(udpipe_url)
textfile = wget.download(text_url)

Загружаем модель *UDPipe*, читаем текстовый файл и обрабатываем его:

In [2]:
from ufal.udpipe import Model, Pipeline

def tag_ud(text='Текст нужно передать функции в виде строки!', modelfile='udpipe_syntagrus.model'):
    model = Model.load(modelfile)
    pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')
    processed = pipeline.process(text) # обрабатываем текст, получаем результат в формате conllu
    output = [l for l in processed.split('\n') if not l.startswith('#')] # пропускаем строки со служебной информацией
    tagged = [w.split('\t')[2].lower() + '_' + w.split('\t')[3] for w in output if w] # извлекаем из обработанного текста лемму и тэг
    tagged_propn = []
    propn  = []
    for t in tagged:
        if t.endswith('PROPN'):
            if propn:
                propn.append(t)
            else:
                propn = [t]
        else:
            if len(propn) > 1:
                name = '::'.join([x.split('_')[0] for x in propn]) + '_PROPN'
                tagged_propn.append(name)
            elif len(propn) == 1:
                tagged_propn.append(propn[0])
            tagged_propn.append(t)
            propn = []
    return tagged_propn

In [3]:
text = open(textfile, 'r', encoding='utf-8').read()
processed_ud = tag_ud(text=text, modelfile=modelfile)
print(processed_ud[:30])

['русский_PROPN', 'соболь_NOUN', '._PUNCT', 'о.::генри_PROPN', 'когда_SCONJ', 'синий_ADJ', ',_PUNCT', 'как_SCONJ', 'ночь_NOUN', ',_PUNCT', 'глаз_NOUN', 'молли_VERB', 'мак-кивер_PROPN', 'класть_VERB', 'малыш::брэди_PROPN', 'на_ADP', 'оба_NUM', 'лопатка_NOUN', ',_PUNCT', 'он_PRON', 'вынужденный_ADJ', 'быть_AUX', 'покидать_VERB', 'ряд_NOUN', 'банда_NOUN', '«дымовый_ADJ', 'труба»_NOUN', '._PUNCT', 'таков_ADJ', 'власть_NOUN']


UDPipe позволяет нам распознавать имена собственные, и несколько идущих подряд имен мы можем склеить в одно.
Вместо UDPipe возможно использовать и Mystem (удобнее использовать [pymystem](https://pypi.python.org/pypi/pymystem3) для Python), хотя Mystem имена собственные не распознает. Для того чтобы работать с последними моделями RusVectōrēs, понадобится сконвертировать тэги Mystem в UPOS.

Сначала нужно установить библиотеку pymystem:

`pip install pymystem3`

Затем импортируем эту библиотеку и анализируем с её помощью текст:

In [4]:
from pymystem3 import Mystem

def tag_mystem(text='Текст нужно передать функции в виде строки!'):  
    m = Mystem()
    processed = m.analyze(text)
    tagged = []
    for w in processed:
        try:
            lemma = w["analysis"][0]["lex"].lower().strip()
            pos = w["analysis"][0]["gr"].split(',')[0]
            pos = pos.split('=')[0].strip()
            tagged.append(lemma.lower() + '_' + pos)
        except KeyError:
            continue # я здесь пропускаю знаки препинания, но вы можете поступить по-другому
    return tagged

In [5]:
processed_mystem = tag_mystem(text=text)
print(processed_mystem[:10])

['русский_S', 'соболь_S', 'о_PR', 'генри_S', 'когда_CONJ', 'синий_A', 'как_CONJ', 'ночь_S', 'глаз_S', 'молль_S']


Как видно, тэги Mystem отличаются от Universal POS-tags, поэтому следующим шагом должна быть их конвертация в Universal Tags. Вы можете воспользоваться вот [этой таблицей конвертации](https://github.com/akutuzov/universal-pos-tags/blob/4653e8a9154e93fe2f417c7fdb7a357b7d6ce333/ru-rnc.map):

In [6]:
import requests
import re

url = 'https://raw.githubusercontent.com/akutuzov/universal-pos-tags/4653e8a9154e93fe2f417c7fdb7a357b7d6ce333/ru-rnc.map'

mapping = {}
r = requests.get(url, stream=True)
for pair in r.text.split('\n'):
    pair = re.sub('\s+', ' ', pair, flags=re.U).split(' ')
    if len(pair) > 1:
        mapping[pair[0]] = pair[1]

print(mapping)

{'COM': 'ADJ', 'APRO': 'DET', 'PART': 'PART', 'PR': 'ADP', 'ADV': 'ADV', 'INTJ': 'INTJ', 'S': 'NOUN', 'V': 'VERB', 'CONJ': 'SCONJ', 'UNKN': 'X', 'ANUM': 'ADJ', 'NUM': 'NUM', 'NONLEX': 'X', 'SPRO': 'PRON', 'ADVPRO': 'ADV', 'A': 'ADJ'}


Теперь усовершенствуем нашу функцию `tag_mystem`:

In [7]:
def tag_mystem(text='Текст нужно передать функции в виде строки!'):  
    m = Mystem()
    processed = m.analyze(text)
    tagged = []
    for w in processed:
        try:
            lemma = w["analysis"][0]["lex"].lower().strip()
            pos = w["analysis"][0]["gr"].split(',')[0]
            pos = pos.split('=')[0].strip()
            if pos in mapping:
                tagged.append(lemma + '_' + mapping[pos]) # здесь мы конвертируем тэги
            else:
                tagged.append(lemma + '_X') # на случай, если попадется тэг, которого нет в маппинге
        except KeyError:
            continue # я здесь пропускаю знаки препинания, но вы можете поступить по-другому
    return tagged

In [8]:
processed_mystem = tag_mystem(text=text)
print(processed_mystem[:10])

['русский_NOUN', 'соболь_NOUN', 'о_ADP', 'генри_NOUN', 'когда_SCONJ', 'синий_ADJ', 'как_SCONJ', 'ночь_NOUN', 'глаз_NOUN', 'молль_NOUN']


Как видим, теперь частеречные тэги в тексте, проанализированном при помощи Mystem, сравнимы с тэгами Unversal POS (хотя сам анализ оказался разным)!

В ходе обработки данных для тренировки моделей мы также удаляли стоп-слова: пунктуацию, служебные части речи, местоимения. Таким образом, наши модели не знают слов "как", "мы" и др. Вы также можете удалить стоп-слова, воспользовавшись, например, [списком стоп-слов в библиотеке NLTK](https://pythonspot.com/nltk-stop-words/).

Итак, в ходе этой части тьюториала мы научились от "сырого текста" приходить к лемматизированному тексту с частеречными тэгами, который уже можно подавать на вход модели! Теперь приступим непосредственно к работе с векторными моделями.

## 2. Работа с векторными моделями при помощи библиотеки Gensim

Прежде чем переходить к работе непосредственно с **RusVectōrēs**, мы посмотрим на то, как работать с дистрибутивными моделями при помощи существующих библиотек. 

Для работы с эмбеддингами слов существуют различные библиотеки: [gensim](https://radimrehurek.com/gensim/), [keras](https://keras.io/), [tensorflow](https://www.tensorflow.org/). Мы будем работать с библиотекой *gensim*, ведь в основе нашего сервера именно она и используется.


***Gensim***  - изначально библиотека для тематического моделирования текстов. Однако помимо различных алгоритмов для *topic modeling* в ней реализованы на python и алгоритмы из тулкита *word2vec* (который в оригинале был написан на C++). Прежде всего, если *gensim* у вас на компьютере не установлен, нужно его установить:

`pip install gensim`

Gensim регулярно обновляется, так что не будет лишним удостовериться, что у вас установлена последняя версия, а при необходимости проапдейтить библиотеку:

`pip install gensim --upgrade` 

или 

`pip install gensim -U`

При подготовке этого тьюториала использовался *gensim* версии 3.4.0.

Поскольку обучение и загрузка моделей могут занимать продолжительное время, иногда бывает полезно вести лог событий. Для этого используется стандартная питоновская библиотека `logging`.

In [9]:
import sys
import gensim, logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

### Работа с моделью

Для своих индивидуальных нужд и экспериментов бывает полезно самому натренировать модель на нужных данных и с нужными параметрами. Но для каких-то общих целей модели уже есть как для русского языка, так и для английского.

Модели для русского скачать можно здесь - https://rusvectores.org/ru/models/

Скачаем модель для русского языка, созданную на основе [Национального корпуса русского языка (НКРЯ)](http://www.ruscorpora.ru/). Вы можете найти её по [этой ссылке](https://rusvectores.org/static/models/rusvectores4/RNC/ruscorpora_upos_skipgram_300_5_2018.vec.gz).

Существуют несколько форматов, в которых могут храниться модели. Во-первых, данные могут храниться в нативном формате *word2vec*, при этом модель может быть бинарной или не бинарной. Для загрузки модели в формате *word2vec* в классе `KeyedVectors` (в котором хранится большинство относящихся к дистрибутивным моделям функций) существует функция `load_word2vec_format`, а бинарность модели можно указать в аргументе `binary` (внизу будет пример). Помимо этого, модель можно хранить и в собственном формате *gensim*, для этого существует класс `Word2Vec` с функцией `load`.

Поскольку модели бывают разных форматов, то для них написаны разные функции загрузки; бывает полезно учитывать это в своем скрипте. Наш код определяет тип модели по её расширению, но вообще файл с моделью может называться как угодно, жестких ограничений для расширения нет. Наша модель, которую мы загрузили с сайта, хранится в небинарном формате word2vec, при этом она сжата при помощи `gzip` (расширение `.gz`), и *gensim* умеет также загружать сжатые модели:

In [10]:
model_url = 'https://rusvectores.org/static/models/rusvectores4/RNC/ruscorpora_upos_skipgram_300_5_2018.vec.gz'
modelfile = wget.download(model_url)
m = 'ruscorpora_upos_skipgram_300_5_2018.vec.gz'
if m.endswith('.vec.gz'):
    model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=False)
elif m.endswith('.bin.gz'):
    model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True)
else:
    model = gensim.models.Word2Vec.load(m)

2018-05-10 22:11:01,499 : INFO : loading projection weights from ruscorpora_upos_skipgram_300_5_2018.vec.gz
2018-05-10 22:13:10,889 : INFO : loaded (195071, 300) matrix from ruscorpora_upos_skipgram_300_5_2018.vec.gz


Для моделей `fasttext` существует отдельный класс и отдельная функция загрузки:

`gensim.models.fasttext.FastText.load()`

Следует иметь в виду, что модели `fasttext` включают в себя несколько файлов, и библиотека не умеет самостоятельно определять, какой из нескольких файлов в архиве необходимо загружать. Следовательно, перед загрузкой скачанный архив с моделью **необходимо распаковать**. Вручную определить необходимый для загрузки файл несложно, чаще всего это файл с расширением `.model` (остальные файлы из архива должны быть в той же папке). Всё это может привести к некоторой путанице (непонятно, когда нужно распаковывать, а когда загружать так), поэтому можно просто запомнить, что модели `fasttext` не такие, как все :)

Существует и альтернативный способ закрузки моделей. Так, некоторые модели можно загрузить прямо из *gensim*, используя метод `gensim.downloader` и репозиторий [**gensim-data**](https://github.com/RaRe-Technologies/gensim-data). В таблице по ссылке можно найти доступные модели и другие ресурсы; наша модель, обученная на НКРЯ, находится под идентификатором `word2vec-ruscorpora-300`. В дальнейшем планируется добавить в репозиторий и другие модели. Репозиторий **gensim-data** регулярно обновляется, однако если вы хотите работать с most up-to-date моделью, лучше всего скачать её с **RusVectōrēs**.

Загрузка моделей при помощи загрузчика gensim выглядит следующим образом:

In [11]:
import gensim.downloader as api

ruscorpora_model = api.load("word2vec-ruscorpora-300")

2018-05-10 22:13:17,155 : INFO : loading projection weights from /home/lizaku/gensim-data/word2vec-ruscorpora-300/word2vec-ruscorpora-300.gz
2018-05-10 22:13:40,775 : INFO : loaded (184973, 300) matrix from /home/lizaku/gensim-data/word2vec-ruscorpora-300/word2vec-ruscorpora-300.gz


Но вернемся к нашей модели, загруженной вручную. Мы можем нормализовать загруженные векторы, чтобы модель занимала меньше места в оперативной памяти.

In [12]:
model.init_sims(replace=True)

2018-05-10 22:13:48,398 : INFO : precomputing L2-norms of word weight vectors


Скажем, нам интересны такие слова (пример для русского языка):

In [13]:
words = ['день_NOUN', 'ночь_NOUN', 'человек_NOUN', 'семантика_NOUN', 'студент_NOUN', 'студент_ADJ']

Попросим у модели 10 ближайших соседей для каждого слова и коэффициент косинусной близости для каждого:

In [14]:
for word in words:
    # есть ли слово в модели? Может быть, и нет
    if word in model:
        print(word)
        # выдаем 10 ближайших соседей слова:
        for i in model.most_similar(positive=[word], topn=10):
            # слово + коэффициент косинусной близости
            print(i[0], i[1])
        print('\n')
    else:
        # Увы!
        print(word + ' is not present in the model')

день_NOUN
день_PROPN 0.6864385604858398
неделя_NOUN 0.6757771372795105
утро_NOUN 0.6553983688354492
днемя_NOUN 0.6455298066139221
месяц_NOUN 0.6271342039108276
вечер_NOUN 0.6199158430099487
воскресенье_NOUN 0.6103948354721069
лабель_PROPN 0.6027745604515076
днямя_NOUN 0.5987443923950195
днями_NOUN 0.5916012525558472


ночь_NOUN
ночь_PROPN 0.8277530670166016
вечер_NOUN 0.7208157777786255
рассвет_NOUN 0.7120692729949951
ночь_ADV 0.677560031414032
утро_NOUN 0.6737046241760254
полночь_NOUN 0.6684714555740356
днемя_NOUN 0.6066733002662659
ночной_ADJ 0.606225848197937
ноченька_NOUN 0.6043546199798584
ночью_NOUN 0.5991744995117188


человек_NOUN
человек_PROPN 0.7707481384277344
человекомя_NOUN 0.6712430715560913
человек-с_NOUN 0.6292698383331299
людей_NOUN 0.596764087677002
людямя_NOUN 0.5873969197273254
женщина_NOUN 0.5705885887145996
человеколо_NOUN 0.5606503486633301
человека_NOUN 0.5553063154220581
поживший_VERB 0.5501028895378113
людьми_NOUN 0.5424647331237793


семантика_NOUN
семантичес

Находим косинусную близость пары слов:

In [15]:
print(model.similarity('человек_NOUN', 'обезьяна_NOUN'))

0.3423996332495893


Найди лишнее!

In [16]:
print(model.doesnt_match('яблоко_NOUN груша_NOUN виноград_NOUN банан_NOUN лимон_NOUN картофель_NOUN'.split()))

картофель_NOUN


Реши пропорцию!

In [17]:
print(model.most_similar(positive=['пицца_NOUN', 'россия_NOUN'], negative=['италия_NOUN'])[0][0])

чипсы_NOUN


## 3. Использование API сервиса RusVectōrēs

Помимо локального использования модели, вы можете также обратиться к RusVectōrēs через API, чтобы использовать наши модели в автоматическом режиме, не скачивая их (скажем, из ваших скриптов). В нашем API имеется две функции:

* получение списка семантически близких слов для заданного слова в заданной модели;
* вычисление коэффициента косинусной близости между парой слов в заданной модели.

Для того чтобы получить список семантически близких слов, необходимо выполнить GET-запрос по адресу в следующем формате:

`https://rusvectores.org/MODEL/WORD/api/FORMAT/`

Разберемся с компонентами этого запроса. `MODEL` - идентификатор модели, к которой мы хотим обратиться. Идентификаторы можно посмотреть в [таблице](https://rusvectores.org/ru/models/) со всеми моделями нашего сервиса. `WORD` - слово, для которого мы хотим узнать соседей. Следует помнить, что частеречный тэг здесь тоже нужен (точнее, вы можете отправлять запросы и без него, но тогда части речи ваших слов сервер определит автоматически - и не всегда правильно). `FORMAT` - формат выходных данных, в настоящий момент это *csv* (с разделением через табуляцию) либо *json*.

Попробуем узнать семантических соседей для первых слов из нашего рассказа. Сначала создадим переменные с параметрами нашего запроса.

In [18]:
print(processed_ud[:15])
MODEL = 'ruscorpora_upos_skipgram_300_5_2018'
FORMAT = 'csv'
WORD = processed_ud[1]

['русский_PROPN', 'соболь_NOUN', '._PUNCT', 'о.::генри_PROPN', 'когда_SCONJ', 'синий_ADJ', ',_PUNCT', 'как_SCONJ', 'ночь_NOUN', ',_PUNCT', 'глаз_NOUN', 'Молли_VERB', 'Мак-Кивер_PROPN', 'класть_VERB', 'малыш::Брэди_PROPN']


In [19]:
def api_neighbor(m, w, f):
    neighbors = {}
    url = '/'.join(['http://rusvectores.org', m, w, 'api', f]) + '/'
    r = requests.get(url=url, stream=True)
    for line in r.text.split('\n'):
        try: # первые две строки в файле -- служебные, их мы пропустим
            word, sim = re.split('\s+', line) # разбиваем строку по одному или более пробелам
            neighbors[word] = sim
        except:
            continue
    return neighbors

In [20]:
print(api_neighbor(MODEL, WORD, FORMAT))

{'чернобурый_ADJ': '0.6514838337898254', 'горностай_NOUN': '0.7106404304504395', 'выдр_NOUN': '0.6077498197555542', 'лисица_NOUN': '0.6174986362457275', 'собольий_ADJ': '0.6177839636802673', 'куница_NOUN': '0.6861780881881714', 'горностай_VERB': '0.6484810709953308', 'соболий_ADJ': '0.6293830871582031', 'горностай_ADJ': '0.650195300579071', 'куний_ADJ': '0.6200308799743652'}


API по умолчанию сообщает 10 ближайших соседей, изменить это количество в данный момент возможности нет.

Теперь рассмотрим вторую функцию, доступную в API - вычисление коэффициента близости между двумя словами.
Запросы для неё должны выполняться в таком виде:

`https://rusvectores.org/MODEL/WORD1__WORD2/api/similarity/`

Здесь переменные - `MODEL` (идентификатор модели, к которой мы обращаемся) и два слова (вместе с их частеречными тэгами). Обратите внимание, что слова разделены **двумя нижними подчеркиваниями**.

In [21]:
def api_similarity(m, w1, w2):
    url = '/'.join(['https://rusvectores.org', m, w1 + '__' + w2, 'api', 'similarity/'])
    r = requests.get(url, stream=True)
    return r.text.split('\t')[0]

In [22]:
MODEL = 'news_upos_cbow_600_2_2018'
api_similarity(MODEL, WORD, 'мех_NOUN')

'0.3407696995115949'

В этом тьюториале мы научились обрабатывать тексты таким образом, чтобы их можно было отдавать в качестве входных данных моделям RusVectōrēs. Мы также рассмотрели основные операции над векторами слов в дистрибутивных семантических моделях и научились обращаться к сервису через API. Надеемся, что данный тьюториал подготовил вас к работе над вашими данными и к новым открытиям, которые можно совершить при помощи дистрибутивной семантики :)